<a href="https://colab.research.google.com/github/siddhant76/Demo/blob/master/first.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!wget -q https://www.stats.govt.nz/assets/Uploads/Injury-statistics/Injury-statistics-work-related-claims-2018/Download-data/injury-statistics-work-related-claims-2018-csv.csv
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"
import findspark
findspark.init()
from pyspark.sql import SparkSession
import numpy as np
from pyspark.sql.functions import col, lit, when
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark
health = spark.read.csv('/content/data.csv',header=True)

In [5]:
data = [('James',None,'Smith','1991-04-01','M',3000),    #None is null
  ('Michael','Rose',None,'2000-05-19','M',4000),
  ('Robert',None,'Williams','1978-09-05','M',4000),
  ('Maria','Anne','Jones','1967-12-01','F',4000),
  ('Jen','Mary','Brown','1980-02-17','F',-1)
]

columns = ["firstname","middlename","lastname","dob","gender","salary"]
df = spark.createDataFrame(data=data, schema = columns)
#df.printSchema()
df.show(truncate=True)

+---------+----------+--------+----------+------+------+
|firstname|middlename|lastname|       dob|gender|salary|
+---------+----------+--------+----------+------+------+
|    James|      null|   Smith|1991-04-01|     M|  3000|
|  Michael|      Rose|    null|2000-05-19|     M|  4000|
|   Robert|      null|Williams|1978-09-05|     M|  4000|
|    Maria|      Anne|   Jones|1967-12-01|     F|  4000|
|      Jen|      Mary|   Brown|1980-02-17|     F|    -1|
+---------+----------+--------+----------+------+------+



In [ ]:
df1=df.filter(df.middlename.isNotNull())
df1.show()

+---------+----------+--------+----------+------+------+
|firstname|middlename|lastname|       dob|gender|salary|
+---------+----------+--------+----------+------+------+
|  Michael|      Rose|    null|2000-05-19|     M|  4000|
|    Maria|      Anne|   Jones|1967-12-01|     F|  4000|
|      Jen|      Mary|   Brown|1980-02-17|     F|    -1|
+---------+----------+--------+----------+------+------+



In [ ]:
from pyspark.sql.functions import col, lit, when

In [ ]:
#df1 = df.withColumn("salary",col("salary") *-8).select("salary")
df1 = df.withColumn("salary",when(df.salary <= 0, df.salary* -100).otherwise(df.salary)).select("salary")
#df1 = df.select("salary")

In [ ]:
df1.show()
median = np.median(df1.collect())
median

+------+
|salary|
+------+
|  3000|
|  4000|
|  4000|
|  4000|
|   100|
+------+



4000.0

In [ ]:
df.show()

+---------+----------+--------+----------+------+------+
|firstname|middlename|lastname|       dob|gender|salary|
+---------+----------+--------+----------+------+------+
|    James|      null|   Smith|1991-04-01|     M|  3000|
|  Michael|      Rose|    null|2000-05-19|     M|  4000|
|   Robert|      null|Williams|1978-09-05|     M|  4000|
|    Maria|      Anne|   Jones|1967-12-01|     F|  4000|
|      Jen|      Mary|   Brown|1980-02-17|     F|    -1|
+---------+----------+--------+----------+------+------+



In [ ]:
df_select = df.filter(df.middlename.isNotNull()).select('firstname','middlename')

In [ ]:
df_select.show()

+---------+----------+
|firstname|middlename|
+---------+----------+
|  Michael|      Rose|
|    Maria|      Anne|
|      Jen|      Mary|
+---------+----------+



In [22]:
df_cha = spark.read.csv('/content/challenge.csv',header=True)
df_cha.show(3)

+--------------+-------+----------------+----------+
|    ip_address|Country|     Domain Name|Bytes_used|
+--------------+-------+----------------+----------+
| 52.81.192.172|  China|odnoklassniki.ru|       463|
|119.239.207.13|  China|        youtu.be|        51|
| 68.69.217.210|  China|       adobe.com|        10|
+--------------+-------+----------------+----------+
only showing top 3 rows



In [27]:
from pyspark.sql.functions import col, lit, when
#create new column based on Country to get IsMexico
df_cha= df_cha.withColumn('IsMexico', when(df_cha.Country == 'Mexico', 'Yes').otherwise('No'))
df_cha.show()


+---------------+--------------+-----------------+----------+--------+
|     ip_address|       Country|      Domain Name|Bytes_used|IsMexico|
+---------------+--------------+-----------------+----------+--------+
|  52.81.192.172|         China| odnoklassniki.ru|       463|      No|
| 119.239.207.13|         China|         youtu.be|        51|      No|
|  68.69.217.210|         China|        adobe.com|        10|      No|
|   7.191.21.223|      Bulgaria|     linkedin.com|       853|      No|
|   211.13.10.68|     Indonesia|          hud.gov|        29|      No|
|   239.80.21.97|      Suriname|       smh.com.au|       218|      No|
|106.214.106.233|       Jamaica|    amazonaws.com|        95|      No|
| 127.242.24.138|         China| surveymonkey.com|       123|      No|
|     99.2.6.139|Czech Republic|     geocities.jp|       322|      No|
|   237.54.11.63|         China|       amazon.com|        83|      No|
| 252.141.157.25|         Japan|      cornell.edu|       374|      No|
|185.2

TypeError: Column is not iterable

In [4]:
import pyspark.sql.functions as sqlfunc
#Based on Ismexico, find the counts of Yes and No
df_GB=df_cha.groupBy('IsMexico').agg(sqlfunc.sum(df_cha.Bytes_used).alias('Total'))

NameError: name 'df_cha' is not defined

In [32]:
df_GB.show()

+--------+--------+
|IsMexico|   Total|
+--------+--------+
|      No|508076.0|
|     Yes|  6293.0|
+--------+--------+



In [37]:
# Group by country and find the distinct IP per country and then sort the count based on IP's
df_IP= df_cha.groupBy('Country').agg(sqlfunc.countDistinct(df_cha.ip_address).alias('IPs_Country'))
df_IP=df_IP.sort(df_IP.IPs_Country.desc())
df_IP.show()

+--------------+-----------+
|       Country|IPs_Country|
+--------------+-----------+
|         China|        172|
|     Indonesia|        114|
|   Philippines|         65|
|        Russia|         56|
|        Brazil|         35|
|        Poland|         31|
|        Sweden|         28|
|         Japan|         25|
|Czech Republic|         23|
|      Portugal|         23|
|        France|         21|
|          Peru|         19|
|      Colombia|         17|
| United States|         15|
|     Argentina|         14|
|       Ukraine|         14|
|        Mexico|         13|
|      Thailand|         12|
|       Nigeria|         11|
|        Canada|         11|
+--------------+-----------+
only showing top 20 rows



In [2]:
from pyspark import SparkConf, SparkContext

# Create a Spark configuration
conf = SparkConf().setAppName("ReadTextFile").setMaster("local[*]")

# Initialize SparkContext
#sc = SparkContext(conf=conf)
ord = spark.sparkContext.textFile('/content/ord')

In [5]:
for i in ord.take(5): print(i)
ordites = spark.sparkContext.textFile('/content/orditems')

1,2013-07-25 00:00:00.0,11599,CLOSED
2,2013-07-25 00:00:00.0,256,PENDING_PAYMENT
3,2013-07-25 00:00:00.0,12111,COMPLETE
4,2013-07-25 00:00:00.0,8827,CLOSED
5,2013-07-25 00:00:00.0,11318,COMPLETE


In [6]:
for i in ordites.take(5): print(i)

1,1,957,1,299.98,299.98
2,2,1073,1,199.99,199.99
3,2,502,5,250.0,50.0
4,2,403,1,129.99,129.99
5,4,897,2,49.98,24.99


In [19]:
ord_map = ord.map(lambda x : (x.split(',')[0],x.split(',')[2]))


In [20]:
for i in ord_map.take(2): print(i)


('1', '11599')
('2', '256')


In [4]:
# Data and schema
data = [(1, 'sid'), (2, 'Nag')]
col = ['Id', 'Name']

# Create DataFrame
df = spark.createDataFrame(data=data, schema=col)

# Show DataFrame
df.show()


+---+----+
| Id|Name|
+---+----+
|  1| sid|
|  2| Nag|
+---+----+



In [5]:
df.createOrReplaceTempView("emp")

In [6]:
spark.sql('select * from emp').show()

+---+----+
| Id|Name|
+---+----+
|  1| sid|
|  2| Nag|
+---+----+



In [10]:
df.createOrReplaceGlobalTempView('dept')
spark.sql(""" Select * from global_temp.dept""").show()

+---+----+
| Id|Name|
+---+----+
|  1| sid|
|  2| Nag|
+---+----+

